In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 파이썬을 이용한 네이버(증권) 주식 정보 크롤링 가이드\n",
    "\n",
    "## 📌 대상 사이트\n",
    "- 네이버 증권: [https://finance.naver.com/sise/sise_market_sum.naver](https://finance.naver.com/sise/sise_market_sum.naver)\n",
    "\n",
    "---\n",
    "\n",
    "## 📌 간단 실습 설명\n",
    "1. 여러 HTTP 요청\n",
    "2. 차단 방지\n",
    "3. 기본 URL과 파라미터 설정\n",
    "4. 사용자 입력\n",
    "5. 데이터 저장을 위한 리스트와 시작 시간 기록\n",
    "6. 페이지별 데이터 수집 반복문\n",
    "7. 데이터프레임 생성 및 전처리\n",
    "8. 데이터 엑셀 파일로 저장 및 서식 적용\n",
    "9. 최종 결과 출력\n",
    "10. GUI 만들기\n",
    "\n",
    "---\n",
    "\n",
    "## 📌 데이터 요청 분석 방법\n",
    "1. 네이버 증권으로 이동 / 국내증시 / 시가총액\n",
    "2. 시가총액(억), PER(배), ROE(%), 외국인비율, 부채총계(억), 매출액증가율 선택 및 적용\n",
    "3. **F12**를 눌러 개발자 도구 실행\n",
    "4. 수집할 CSS 선택자 확인\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 필요한 도구 준비하기\n",
    "---\n",
    "### 💡 설명\n",
    "#### requests: 인터넷에서 정보를 가져오는 도구\n",
    "#### BeautifulSoup: 웹페이지에서 원하는 정보만 골라내는 도구\n",
    "#### pandas: 데이터를 표 형태로 정리하는 도구\n",
    "#### time: 시간을 측정하고 대기하는 도구\n",
    "#### HTTPAdapter, Retry: 인터넷 연결이 불안정할 때 재시도하는 도구"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "pip install requests beautifulsoup4 pandas time "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import pandas as pd\n",
    "import time\n",
    "from requests.adapters import HTTPAdapter\n",
    "from urllib3.util.retry import Retry"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 안정적인 연결 설정하기\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 네이버 금융 사이트에 안정적으로 접속하기 위한 설정입니다:\n",
    "#### 세션 설정: 같은 웹사이트에 여러 번 접속할 때 효율적으로 연결하는 방법\n",
    "#### 재시도 설정: 연결이 실패하면 최대 5번까지 다시 시도\n",
    "#### 헤더 설정: 일반 웹브라우저처럼 보이게 하여 차단되지 않도록 함 (마치 가면을 쓰는 것과 같음)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "session = requests.Session()\n",
    "retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])\n",
    "session.mount('https://', HTTPAdapter(max_retries=retries))\n",
    "\n",
    "headers = {\n",
    "    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'\n",
    "}"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 사용자 입력 받기\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 사용자에게 몇 페이지의 주식 정보를 수집할지 물어보는 부분입니다:\n",
    "#### 빈 리스트 data는 수집한 정보를 담을 그릇\n",
    "#### start_time은 작업 시작 시간을 기록하여 나중에 총 소요 시간을 계산\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "pages = int(input(\"수집할 페이지의 수를 입력해 주세요\"))\n",
    "\n",
    "data = []\n",
    "start_time = time.time()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 데이터 수집 과정\n",
    "---\n",
    "### 💡 설명\n",
    "#### url: 네이버 금융 사이트 주소와 원하는 정보 항목을 지정\n",
    "####    ㄴfieldIds: 수집할 항목들 (시가총액, 부채총계, 매출증가율, 외국인비율, PER, ROE)    \n",
    "#### response: 웹사이트에서 받아온 응답\n",
    "#### timeout=10: 10초 안에 응답이 없으면 다시 시도\n",
    "#### raise_for_status(): 오류가 발생하면 알려줌"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for page in range(1, pages+1):\n",
    "    try:\n",
    "        url = f\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe\"\n",
    "        # 효율적인 URL 구성 및 요청\n",
    "        response = session.get(\n",
    "            url,\n",
    "            headers=headers,\n",
    "            timeout=10\n",
    "        )\n",
    "        response.raise_for_status()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### 차이점 비교해보기"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=1&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe\"\n",
    "\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=2&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe\"\n",
    "\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=2&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 웹페이지에서 정보 추출하기\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 웹페이지에서 원하는 정보만 골라내는 과정입니다:\n",
    "#### BeautifulSoup: 웹페이지의 내용을 분석하는 도구   \n",
    "#### soup.select: 표(table)에서 주식 정보가 있는 행(tr)들을 선택\n",
    "#### 각 행에서 필요한 정보를 추출:\n",
    "#### ㄴname: 종목명 (2번째 열)\n",
    "#### ㄴmarket_sum: 시가총액 (7번째 열)\n",
    "#### ㄴdebt_total: 부채총계 (8번째 열)\n",
    "#### ㄴsales_increasing_rate: 매출증가율 (9번째 열)\n",
    "#### ㄴfrgn_rate: 외국인비율 (10번째 열)\n",
    "#### ㄴper: PER (주가수익비율) (11번째 열)\n",
    "#### ㄴroe: ROE (자기자본이익률) (12번째 열)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "soup = BeautifulSoup(response.content, 'html.parser')\n",
    "trs = soup.select(\"#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']\")\n",
    "        \n",
    "for tr in trs:\n",
    "    name = tr.select_one('td:nth-child(2)').text\n",
    "    market_sum = tr.select_one('td:nth-child(7)').text\n",
    "    debt_total = tr.select_one('td:nth-child(8)').text\n",
    "    sales_increasing_rate = tr.select_one('td:nth-child(9)').text\n",
    "    frgn_rate = tr.select_one('td:nth-child(10)').text\n",
    "    per = tr.select_one('td:nth-child(11)').text\n",
    "    roe = tr.select_one('td:nth-child(12)').text"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) \n",
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)\n",
    "\n",
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)\n",
    "#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']\n",
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)\n",
    "#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(8)\n",
    "// 위치 기반 선택:\n",
    "\n",
    "“#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)”는 tbody 안의 두 번째 tr 요소를 선택합니다.\n",
    "\n",
    "// 속성 기반 선택:\n",
    "\n",
    "“#contentarea > div.box_type_l > table.type_2 > tbody > tr[onmouseover='mouseOver(this)']”는 onmouseover 속성의 값이 \"mouseOver(this)\"인 tr 요소를 선택합니다.\n",
    "\n",
    "# 페이지 구조상 두 번째 tr 요소에만 onmouseover 속성이 지정되어 있다면, \n",
    "# 두 선택자는 자연스럽게 같은 요소를 가리키게 됩니다. \n",
    "# 이렇게 하면, 위치에 기반한 선택과 이벤트나 인터랙션을 \n",
    "# 위해 속성에 기반한 선택을 동시에 활용할 수 있습니다."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 데이터 전처리 및 저장\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 수집한 데이터를 정리하는 과정입니다:\n",
    "#### if market_sum != 'N/A' and ...: 모든 정보가 있는 종목만 선택 ('N/A'는 정보 없음을 의미)   \n",
    "#### replace(',', ''): 숫자에서 쉼표(,) 제거 (예: '1,000' → '1000') -> 이 작업을 하지 않으면 숫자를 텍스트로 인식해버림.\n",
    "#### float(): 문자열을 숫자로 변환\n",
    "#### print(): 수집한 정보를 화면에 출력\n",
    "#### data.append(): 수집한 정보를 리스트에 추가"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':\n",
    "    market_sum = float(market_sum.replace(',', ''))\n",
    "    debt_total = float(debt_total.replace(',', ''))\n",
    "    sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))\n",
    "    frgn_rate = float(frgn_rate.replace(',', ''))\n",
    "    per = float(per.replace(',', ''))\n",
    "    roe = float(roe.replace(',', ''))\n",
    "    print(name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe)\n",
    "    # 엑셀 추가\n",
    "    data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 서버 부하 방지 및 오류 처리\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 안정적인 데이터 수집을 위한 설정입니다:\n",
    "#### time.sleep(1.5): 1.5초 동안 대기 (너무 빠르게 요청하면 차단될 수 있음)\n",
    "#### except Exception as e: 오류가 발생해도 프로그램이 중단되지 않고 계속 실행\n",
    "#### continue: 오류가 발생해도 다음 페이지로 넘어감"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "time.sleep(1.5)\n",
    "\n",
    "except Exception as e:\n",
    "    print(f\"페이지 {page} 처리 중 오류 발생: {str(e)}\")\n",
    "    continue"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 데이터프레임 생성 및 엑셀 저장 \n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 수집한 데이터를 엑셀 파일로 저장하는 과정입니다:\n",
    "#### pd.DataFrame: 수집한 데이터를 표 형태로 변환\n",
    "#### columns=[\"종목명\", ...]: 각 열의 이름 지정\n",
    "#### dropna(): 빈 값이 있는 행 제거\n",
    "#### timestamp: 현재 날짜와 시간을 파일명에 포함 (예: 'stock_data_20240320_153045.xlsx')\n",
    "#### pd.ExcelWriter: 엑셀 파일 생성\n",
    "#### df.to_excel: 데이터프레임을 엑셀 파일로 저장\n",
    "#### number_format: 숫자 형식 설정 (소수점 두 자리까지 표시)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = pd.DataFrame(data, columns=[\"종목명\", \"시가총액\", \"부채총계\", \"매출증가율\", \"외국인비율\", \"PER\", \"ROE\"]).dropna()\n",
    "\n",
    "timestamp = time.strftime(\"%Y%m%d_%H%M%S\")\n",
    "with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:\n",
    "    df.to_excel(writer, index=False)\n",
    "    workbook = writer.book\n",
    "    worksheet = writer.sheets['Sheet1']\n",
    "    number_format = workbook.add_format({'num_format': '#,##0.00'})\n",
    "    worksheet.set_column('B:E', 15, number_format)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 작업 완료 메시지\n",
    "---\n",
    "### 💡 설명\n",
    "#### 이 부분은 작업이 완료되었음을 알려주는 메시지입니다:\n",
    "#### len(df): 수집한 종목 수\n",
    "#### time.time()-start_time: 작업에 소요된 시간 (초)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(f\"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ 최종 코드\n",
    "---\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "삼성전자 3249881.0 1123399.0 16.2 50.26 11.09 9.03\n",
      "SK하이닉스 1447997.0 459395.0 102.02 55.26 7.32 31.06\n",
      "LG에너지솔루션 815490.0 293402.0 -24.08 4.47 -80.04 -4.93\n",
      "삼성바이오로직스 767967.0 64316.0 23.08 13.52 70.89 10.45\n",
      "현대차 415063.0 2195225.0 7.73 37.03 4.3 12.43\n",
      "셀트리온 400168.0 27917.0 -4.71 22.67 209.76 5.07\n",
      "기아 394094.0 369156.0 7.66 39.19 4.06 19.09\n",
      "NAVER 339055.0 114998.0 17.65 48.57 20.64 4.41\n",
      "한화에어로스페이스 303115.0 148587.0 32.56 45.31 54.49 25.6\n",
      "KB금융 302623.0 6568648.0 -6.72 75.42 6.64 8.44\n",
      "HD현대중공업 262768.0 119262.0 32.26 10.0 91.9 0.47\n",
      "POSCO홀딩스 259441.0 412815.0 -9.0 29.3 20.09 3.18\n",
      "현대모비스 230628.0 204787.0 -3.41 41.61 5.7 9.35\n",
      "신한지주 227306.0 6354735.0 24.57 58.86 5.03 8.36\n",
      "메리츠금융지주 226003.0 921387.0 -12.88 15.65 10.07 28.11\n",
      "고려아연 220490.0 24041.0 -13.5 12.25 35.23 5.72\n",
      "한화오션 217554.0 96326.0 52.43 11.5 88.75 6.33\n",
      "SK이노베이션 209334.0 508155.0 -0.98 14.55 -10.2 1.22\n",
      "삼성물산 203292.0 247319.0 0.5 27.57 9.74 6.83\n",
      "카카오 195661.0 113214.0 11.15 27.65 -21.78 -10.26\n",
      "삼성화재 186894.0 688746.0 6.19 53.6 9.77 12.73\n",
      "HMM 184578.0 42726.0 -54.79 9.8 4.64 4.59\n",
      "LG화학 177187.0 365285.0 8.37 32.15 69.39 4.2\n",
      "하나금융지주 173781.0 5517443.0 -0.79 67.19 4.81 9.01\n",
      "삼성생명 168200.0 2703176.0 -10.03 21.57 6.76 4.95\n",
      "두산에너빌리티 168147.0 137993.0 14.06 24.04 205.08 0.78\n",
      "크래프톤 165523.0 8816.0 3.05 41.6 20.79 11.16\n",
      "HD한국조선해양 150393.0 198725.0 23.09 31.0 24.78 2.26\n",
      "삼성SDI 146468.0 190301.0 -22.6 33.29 25.01 3.13\n",
      "한국전력 142837.0 2024502.0 23.8 15.94 3.85 -12.63\n",
      "LG전자 134518.0 367423.0 0.91 30.91 19.14 3.69\n",
      "삼성중공업 126368.0 121842.0 34.73 30.6 -202.25 -4.22\n",
      "기업은행 124638.0 4166105.0 4.05 13.88 4.54 8.79\n",
      "KT 123113.0 241488.0 2.83 49.0 10.58 6.05\n",
      "HD현대일렉트릭 123101.0 18531.0 28.43 33.47 24.08 27.71\n",
      "SK스퀘어 122864.0 21535.0 -91.81 51.95 7.4 -8.04\n",
      "SK텔레콤 120068.0 178908.0 1.75 42.2 10.65 9.63\n",
      "우리금융지주 118740.0 4646074.0 -0.91 45.63 4.38 8.3\n",
      "KT&G 116203.0 45668.0 0.79 44.44 10.61 12.66\n",
      "포스코퓨처엠 112632.0 37231.0 44.16 9.35 -526.81 1.19\n",
      "카카오뱅크 109466.0 483705.0 55.32 16.81 25.39 6.0\n",
      "LG 106650.0 32366.0 3.61 35.15 11.4 4.89\n",
      "포스코인터내셔널 103970.0 99930.0 -12.78 6.5 15.41 13.36\n",
      "현대로템 102921.0 32763.0 22.0 30.49 25.3 21.85\n",
      "삼성전기 102554.0 37765.0 15.76 32.64 15.69 8.16\n",
      "SK 99329.0 1290912.0 -0.64 23.98 -42.92 -3.67\n",
      "하이브 97049.0 22358.0 22.63 19.63 -514.35 6.58\n",
      "유한양행 95288.0 7123.0 4.68 19.14 57.64 6.74\n",
      "삼성에스디에스 92157.0 33455.0 -22.97 18.59 13.0 8.22\n",
      "현대글로비스 91125.0 69411.0 -4.81 49.99 7.32 14.52\n",
      "한미반도체 87339.0 1519.0 -51.46 12.22 48.39 55.54\n",
      "대한항공 85795.0 205766.0 14.3 18.56 7.93 11.46\n",
      "SK바이오팜 82620.0 4022.0 44.16 11.96 143.93 -10.96\n",
      "한국항공우주 79930.0 55196.0 37.05 35.93 27.58 14.81\n",
      "DB손해보험 68818.0 497367.0 4.38 46.51 3.44 15.66\n",
      "S-Oil 68113.0 125386.0 -15.83 75.02 71.09 10.83\n",
      "한화시스템 66594.0 23291.0 12.12 6.51 50.0 16.87\n",
      "아모레퍼시픽 66389.0 10138.0 -11.15 26.39 11.35 3.72\n",
      "삼양식품 66291.0 5935.0 31.23 15.9 29.17 24.81\n",
      "HD현대마린솔루션 62671.0 4167.0 7.25 4.28 37.0 71.59\n",
      "LIG넥스원 61710.0 27634.0 3.95 28.23 31.01 17.61\n",
      "코웨이 60228.0 22132.0 2.86 61.57 10.88 19.03\n",
      "LS ELECTRIC 59370.0 20088.0 25.27 23.89 27.22 12.61\n",
      "HD현대 59245.0 447510.0 0.79 20.85 23.46 3.57\n",
      "두산 56512.0 170801.0 12.56 13.38 -22.72 -22.74\n",
      "한진칼 55413.0 9310.0 37.63 22.07 12.38 14.79\n",
      "미래에셋증권 54961.0 1169384.0 6.24 13.65 13.23 2.97\n",
      "에코프로머티 51746.0 3051.0 43.18 25.68 -153.31 0.91\n",
      "LG생활건강 51540.0 17199.0 0.1 28.45 30.91 3.44\n",
      "두산밥캣 51428.0 44169.0 13.19 32.96 7.84 16.79\n",
      "LG씨엔에스 50574.0 21724.0 12.79 0.48 11.86 19.05\n",
      "삼성카드 49530.0 206706.0 5.53 6.43 6.97 7.61\n",
      "한국타이어앤테크놀로지 48807.0 31338.0 6.5 37.49 3.92 7.85\n",
      "SKC 47828.0 45079.0 -34.19 15.54 -15.49 -17.57\n",
      "NH투자증권 47814.0 490845.0 -7.63 17.35 7.64 7.5\n",
      "LG유플러스 46674.0 113438.0 3.36 35.41 8.76 7.49\n",
      "LG디스플레이 45650.0 247868.0 24.77 20.67 -1.68 -37.21\n",
      "포스코DX 42874.0 4284.0 28.91 3.92 54.02 21.08\n",
      "오리온 42778.0 5659.0 1.36 28.64 11.15 13.91\n",
      "카카오페이 41875.0 20317.0 18.03 37.9 -213.01 0.14\n",
      "한국금융지주 41683.0 874561.0 -7.79 39.43 5.08 8.81\n",
      "HD현대미포 40981.0 28939.0 8.67 17.18 -50.2 -6.96\n",
      "두산로보틱스 40642.0 176.0 17.99 3.86 -300.0 -6.59\n",
      "LG이노텍 40565.0 60243.0 2.89 24.03 9.03 8.92\n",
      "삼성증권 40364.0 498852.0 -2.36 30.92 5.43 8.54\n",
      "효성중공업 39723.0 35371.0 22.52 15.16 25.32 11.29\n",
      "현대제철 38899.0 157228.0 -5.22 19.52 -20.16 2.44\n",
      "CJ제일제당 38765.0 178263.0 -3.51 22.39 9.14 5.55\n",
      "현대건설 37527.0 132590.0 39.61 20.16 8.79 6.77\n",
      "현대오토에버 36968.0 12507.0 11.27 1.86 24.05 9.01\n",
      "삼성E&A 36358.0 45031.0 5.68 43.05 4.8 24.45\n",
      "강원랜드 36199.0 7708.0 9.28 12.39 8.65 9.61\n",
      "한화솔루션 35066.0 154824.0 1.2 13.7 -2.28 -2.48\n",
      "엔씨소프트 34643.0 11408.0 -30.79 36.78 27.36 6.58\n",
      "한화 34256.0 1765394.0 4.42 14.39 -70.74 3.52\n",
      "GS 34193.0 168232.0 -9.11 17.78 5.21 10.05\n",
      "넷마블 33737.0 28368.0 -6.41 25.5 172.15 -4.9\n",
      "BNK금융지주 33558.0 1351099.0 27.62 41.63 4.93 6.38\n",
      "한국가스공사 33094.0 474286.0 -13.86 10.08 21.44 -7.68\n",
      "LS 32780.0 114954.0 39.96 15.25 6.6 9.63\n",
      "JB금융지주 32402.0 582880.0 31.25 36.1 4.99 12.23\n",
      "시프트업 32259.0 229.0 158.1 3.27 25.94 86.44\n",
      "SK바이오사이언스 32162.0 1486.0 -19.1 5.36 -72.91 1.3\n",
      "롯데케미칼 32039.0 132438.0 -10.46 22.35 -4.19 -0.33\n",
      "한미약품 31131.0 7985.0 11.97 14.68 19.38 16.01\n",
      "키움증권 30964.0 471423.0 6.74 25.92 6.99 9.27\n",
      "CJ 30723.0 293109.0 1.05 14.31 29.48 3.64\n",
      "금호석유 30451.0 21457.0 -20.73 17.81 9.12 7.77\n",
      "한온시스템 27626.0 67359.0 10.8 5.56 -21.2 2.21\n",
      "엘앤에프 27404.0 22415.0 19.47 18.02 -5.02 -16.35\n",
      "F&F 27160.0 6874.0 9.38 11.92 7.65 38.43\n",
      "동서 25523.0 697.0 -0.17 4.19 16.23 9.52\n",
      "에이피알 25138.0 904.0 31.71 16.12 28.6 54.88\n",
      "KCC 24571.0 79059.0 -7.18 11.11 5.3 4.13\n",
      "이수페타시스 24476.0 3597.0 5.04 14.56 40.35 19.51\n",
      "한전기술 24384.0 3077.0 7.88 9.64 61.82 5.99\n",
      "롯데지주 24339.0 134876.0 7.43 7.49 -5.82 -0.22\n",
      "농심 23783.0 7939.0 9.0 18.4 15.34 7.26\n",
      "한화생명 23233.0 1324268.0 2.35 10.0 3.96 5.43\n",
      "이마트 22942.0 196096.0 0.48 19.11 -44.08 -0.79\n",
      "에스원 22344.0 5816.0 6.19 55.88 11.38 12.14\n",
      "대한전선 22262.0 9253.0 16.06 5.88 18.09 7.83\n",
      "SK가스 22153.0 34935.0 -13.31 7.39 13.43 12.85\n",
      "현대해상 22126.0 380326.0 3.67 38.3 2.52 7.81\n",
      "아시아나항공 21917.0 122064.0 22.83 2.73 8.96 16.61\n",
      "휠라홀딩스 21604.0 24262.0 -5.1 43.12 21.05 2.25\n",
      "더존비즈온 21359.0 4380.0 16.19 12.49 45.27 8.19\n",
      "CJ대한통운 21170.0 53145.0 -2.99 14.32 9.18 6.27\n",
      "현대엘리베이 20836.0 21639.0 22.2 28.35 31.28 26.15\n",
      "영원무역 20671.0 17092.0 -7.84 29.38 4.9 17.03\n",
      "코스맥스 20293.0 11985.0 11.09 33.88 25.79 12.11\n",
      "DB하이텍 20135.0 3033.0 -30.86 25.65 8.62 15.65\n",
      "제일기획 19937.0 15917.0 -2.71 22.86 10.68 14.99\n",
      "팬오션 19699.0 31406.0 -32.08 15.31 6.22 5.32\n",
      "한화엔진 19527.0 9127.0 11.8 9.16 51.66 -0.19\n",
      "HL만도 19276.0 39305.0 11.67 26.05 15.68 6.18\n",
      "SK아이이테크놀로지 19143.0 16722.0 10.89 6.32 -26.22 3.58\n",
      "롯데쇼핑 19010.0 198082.0 -5.95 8.61 -59.73 1.83\n",
      "한미사이언스 18876.0 4387.0 19.3 2.02 18.19 15.23\n",
      "HD현대인프라코어 18765.0 26051.0 -2.03 11.97 31.52 13.37\n",
      "한전KPS 18652.0 3247.0 7.33 16.62 10.19 13.08\n",
      "산일전기 18480.0 654.0 81.33 9.13 6.89 47.0\n",
      "한올바이오파마 18075.0 535.0 22.65 5.85 -288.33 1.98\n",
      "BGF리테일 18044.0 21446.0 7.6 29.0 9.48 19.23\n",
      "DL이앤씨 17490.0 45753.0 6.59 31.89 16.78 4.12\n",
      "아모레G 17322.0 11063.0 -10.54 9.04 7.56 3.57\n",
      "풍산 17235.0 16947.0 -5.67 20.83 7.41 8.22\n",
      "코스모신소재 16093.0 1939.0 29.65 8.76 61.64 7.41\n",
      "오뚜기 16051.0 14329.0 8.52 12.15 10.26 8.55\n",
      "코리안리 15780.0 88134.0 -9.44 27.14 5.96 9.51\n",
      "에스엘 15630.0 12266.0 15.91 16.53 4.21 18.83\n",
      "GS건설 15473.0 128221.0 9.25 24.9 -37.82 -10.54\n",
      "호텔신라 15346.0 25295.0 10.62 14.18 -25.42 -6.5\n",
      "신세계 15299.0 86971.0 3.36 14.23 14.19 2.54\n",
      "한국콜마 15296.0 15908.0 15.54 34.31 156.52 0.79\n",
      "씨에스윈드 15287.0 18882.0 10.57 13.33 13.54 2.24\n",
      "한국앤컴퍼니 15266.0 6243.0 -0.57 9.36 4.12 4.76\n",
      "OCI홀딩스 15033.0 21416.0 -4.31 20.39 9.44 18.69\n",
      "한솔케미칼 15019.0 4095.0 -12.85 33.13 12.72 13.1\n",
      "DGB금융지주 14952.0 870762.0 17.87 41.02 6.93 6.73\n",
      "녹십자 14748.0 11035.0 -4.95 19.63 -78.24 -2.03\n",
      "대우건설 14505.0 71812.0 11.79 11.89 4.42 13.19\n",
      "동원산업 14166.0 45894.0 -0.86 1.95 29.33 8.62\n",
      "금호타이어 14047.0 34319.0 13.55 8.3 4.19 13.2\n",
      "대웅제약 13927.0 8580.0 7.44 8.68 13.45 17.7\n",
      "HD현대건설기계 13638.0 15259.0 8.8 15.43 28.74 9.17\n",
      "HDC현대산업개발 13610.0 40083.0 27.06 12.61 8.21 5.87\n",
      "총 167개 데이터 수집 완료 | 소요 시간: 7.94초\n"
     ]
    }
   ],
   "source": [
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import pandas as pd\n",
    "import time\n",
    "from requests.adapters import HTTPAdapter\n",
    "from urllib3.util.retry import Retry\n",
    "\n",
    "# 세션 설정으로 성능 향상 및 안정성 확보\n",
    "session = requests.Session()\n",
    "retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])\n",
    "session.mount('https://', HTTPAdapter(max_retries=retries))\n",
    "\n",
    "# 웹 크롤링 차단 방지를 위한 헤더 설정\n",
    "headers = {\n",
    "    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'\n",
    "}\n",
    "\n",
    "# 사용자 입력 받기\n",
    "pages = int(input(\"수집할 페이지의 수를 입력해 주세요\"))\n",
    "\n",
    "data = []\n",
    "start_time = time.time()\n",
    "\n",
    "#만약 내가 원하는 체크항목이 있다면, f12를 누르고 체크칸에 value 값을 찾아서fieldIds에 넣어준다.\n",
    "for page in range(1, pages+1):\n",
    "    try:\n",
    "        url = f\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe\"\n",
    "        # 효율적인 URL 구성 및 요청\n",
    "        response = session.get(\n",
    "            url,\n",
    "            headers=headers,\n",
    "            timeout=10\n",
    "        )\n",
    "        response.raise_for_status()\n",
    "        \n",
    "\n",
    "        soup = BeautifulSoup(response.content, 'html.parser')\n",
    "        trs = soup.select(\"#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']\")\n",
    "        \n",
    "        for tr in trs:\n",
    "            name = tr.select_one('td:nth-child(2)').text\n",
    "            market_sum = tr.select_one('td:nth-child(7)').text\n",
    "            debt_total = tr.select_one('td:nth-child(8)').text\n",
    "            sales_increasing_rate = tr.select_one('td:nth-child(9)').text\n",
    "            frgn_rate = tr.select_one('td:nth-child(10)').text\n",
    "            per = tr.select_one('td:nth-child(11)').text\n",
    "            roe = tr.select_one('td:nth-child(12)').text\n",
    "\n",
    "            # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리\n",
    "            if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':\n",
    "                market_sum = float(market_sum.replace(',', ''))\n",
    "                debt_total = float(debt_total.replace(',', ''))\n",
    "                sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))\n",
    "                frgn_rate = float(frgn_rate.replace(',', ''))\n",
    "                per = float(per.replace(',', ''))\n",
    "                roe = float(roe.replace(',', ''))\n",
    "                print(name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe)\n",
    "                # 엑셀 추가\n",
    "                data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])\n",
    "                \n",
    "        # 요청 간격 추가로 서버 부하 방지\n",
    "        time.sleep(1.5)\n",
    "\n",
    "    except Exception as e:\n",
    "        print(f\"페이지 {page} 처리 중 오류 발생: {str(e)}\")\n",
    "        continue\n",
    "\n",
    "# 데이터프레임 처리 \n",
    "df = pd.DataFrame(data, columns=[\"종목명\", \"시가총액(억)\", \"부채총계(억)\", \"매출증가율\", \"외국인비율\", \"PER(배)\", \"ROE(%)\"]).dropna()\n",
    "\n",
    "\n",
    "# 파일 저장 최적화\n",
    "timestamp = time.strftime(\"%Y%m%d_%H%M%S\")\n",
    "with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:\n",
    "    df.to_excel(writer, index=False)\n",
    "    # 엑셀 서식 자동 적용\n",
    "    workbook = writer.book\n",
    "    worksheet = writer.sheets['Sheet1']\n",
    "    number_format = workbook.add_format({'num_format': '#,##0.00'})\n",
    "    worksheet.set_column('B:E', 15, number_format)\n",
    "\n",
    "print(f\"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초\")\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 저는 다른 항목을 선택하고 싶어요!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "삼성전자 17766220.0 54900.0\n",
      "SK하이닉스 4040807.0 198800.0\n",
      "LG에너지솔루션 188759.0 348000.0\n",
      "삼성바이오로직스 55769.0 1079000.0\n",
      "현대차 349139.0 198100.0\n",
      "셀트리온 456404.0 186800.0\n",
      "기아 1132010.0 99000.0\n",
      "삼성전자우 1416552.0 45550.0\n",
      "NAVER 407265.0 213500.0\n",
      "한화에어로스페이스 720529.0 664000.0\n",
      "KB금융 1423592.0 76900.0\n",
      "HD현대중공업 347594.0 296000.0\n",
      "POSCO홀딩스 738356.0 313500.0\n",
      "현대모비스 172564.0 247500.0\n",
      "신한지주 1182678.0 45150.0\n",
      "메리츠금융지주 224782.0 118400.0\n",
      "고려아연 170654.0 1064000.0\n",
      "한화오션 9319988.0 71000.0\n",
      "SK이노베이션 388693.0 138100.0\n",
      "삼성물산 281003.0 119600.0\n",
      "카카오 1959875.0 44050.0\n",
      "삼성화재 82655.0 393000.0\n",
      "HMM 2139479.0 20900.0\n",
      "LG화학 308850.0 250500.0\n",
      "하나금융지주 1076352.0 60500.0\n",
      "삼성생명 210248.0 84100.0\n",
      "두산에너빌리티 9404649.0 26250.0\n",
      "크래프톤 103673.0 345500.0\n",
      "HD한국조선해양 436894.0 212500.0\n",
      "삼성SDI 375041.0 213000.0\n",
      "한국전력 862335.0 22250.0\n",
      "LG전자 857936.0 82200.0\n",
      "삼성중공업 48695435.0 14360.0\n",
      "기업은행 960662.0 15630.0\n",
      "KT 292793.0 48800.0\n",
      "HD현대일렉트릭 381001.0 341500.0\n",
      "SK스퀘어 234850.0 91900.0\n",
      "SK텔레콤 371308.0 55800.0\n",
      "우리금융지주 1782639.0 15980.0\n",
      "KT&G 259096.0 95200.0\n",
      "포스코퓨처엠 385808.0 145400.0\n",
      "카카오뱅크 275067.0 22900.0\n",
      "LG 243061.0 67800.0\n",
      "포스코인터내셔널 2075685.0 59100.0\n",
      "현대로템 5164291.0 94200.0\n",
      "삼성전기 197848.0 137300.0\n",
      "SK 145049.0 136900.0\n",
      "하이브 211428.0 232500.0\n",
      "유한양행 446514.0 118800.0\n",
      "삼성에스디에스 121096.0 119000.0\n",
      "삼성전자 17766220.0 54900.0\n",
      "SK하이닉스 4040807.0 198800.0\n",
      "LG에너지솔루션 188759.0 348000.0\n",
      "삼성바이오로직스 55769.0 1079000.0\n",
      "현대차 349139.0 198100.0\n",
      "셀트리온 456404.0 186800.0\n",
      "기아 1132010.0 99000.0\n",
      "삼성전자우 1416552.0 45550.0\n",
      "NAVER 407265.0 213500.0\n",
      "한화에어로스페이스 720529.0 664000.0\n",
      "KB금융 1423592.0 76900.0\n",
      "HD현대중공업 347594.0 296000.0\n",
      "POSCO홀딩스 738356.0 313500.0\n",
      "현대모비스 172564.0 247500.0\n",
      "신한지주 1182678.0 45150.0\n",
      "메리츠금융지주 224782.0 118400.0\n",
      "고려아연 170654.0 1064000.0\n",
      "한화오션 9319988.0 71000.0\n",
      "SK이노베이션 388693.0 138100.0\n",
      "삼성물산 281003.0 119600.0\n",
      "카카오 1959875.0 44050.0\n",
      "삼성화재 82655.0 393000.0\n",
      "HMM 2139479.0 20900.0\n",
      "LG화학 308850.0 250500.0\n",
      "하나금융지주 1076352.0 60500.0\n",
      "삼성생명 210248.0 84100.0\n",
      "두산에너빌리티 9404649.0 26250.0\n",
      "크래프톤 103673.0 345500.0\n",
      "HD한국조선해양 436894.0 212500.0\n",
      "삼성SDI 375041.0 213000.0\n",
      "한국전력 862335.0 22250.0\n",
      "LG전자 857936.0 82200.0\n",
      "삼성중공업 48695435.0 14360.0\n",
      "기업은행 960662.0 15630.0\n",
      "KT 292793.0 48800.0\n",
      "HD현대일렉트릭 381001.0 341500.0\n",
      "SK스퀘어 234850.0 91900.0\n",
      "SK텔레콤 371308.0 55800.0\n",
      "우리금융지주 1782639.0 15980.0\n",
      "KT&G 259096.0 95200.0\n",
      "포스코퓨처엠 385808.0 145400.0\n",
      "카카오뱅크 275067.0 22900.0\n",
      "LG 243061.0 67800.0\n",
      "포스코인터내셔널 2075685.0 59100.0\n",
      "현대로템 5164291.0 94200.0\n",
      "삼성전기 197848.0 137300.0\n",
      "SK 145049.0 136900.0\n",
      "하이브 211428.0 232500.0\n",
      "유한양행 446514.0 118800.0\n",
      "삼성에스디에스 121096.0 119000.0\n",
      "총 100개 데이터 수집 완료 | 소요 시간: 3.70초\n"
     ]
    }
   ],
   "source": [
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import pandas as pd\n",
    "import time\n",
    "from requests.adapters import HTTPAdapter\n",
    "from urllib3.util.retry import Retry\n",
    "\n",
    "# 세션 설정으로 성능 향상 및 안정성 확보\n",
    "session = requests.Session()\n",
    "retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])\n",
    "session.mount('https://', HTTPAdapter(max_retries=retries))\n",
    "\n",
    "# 웹 크롤링 차단 방지를 위한 헤더 설정\n",
    "headers = {\n",
    "    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'\n",
    "}\n",
    "\n",
    "# 사용자 입력 받기\n",
    "pages = int(input(\"수집할 페이지의 수를 입력해 주세요세요\"))\n",
    "\n",
    "data = []\n",
    "start_time = time.time()\n",
    "\n",
    "#만약 내가 원하는 체크항목이 있다면, f12를 누르고 체크칸에 value 값을 찾아서fieldIds에 넣어준다.\n",
    "for page in range(1, pages+1):\n",
    "    try:\n",
    "        url = f\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=1&fieldIds=quant&fieldIds=ask_buy\"\n",
    "        # 효율적인 URL 구성 및 요청\n",
    "        response = session.get(\n",
    "            url,\n",
    "            headers=headers,\n",
    "            timeout=10\n",
    "        )\n",
    "        response.raise_for_status()\n",
    "\n",
    "        soup = BeautifulSoup(response.content, 'html.parser')\n",
    "        trs = soup.select(\"#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']\")\n",
    "        \n",
    "        for tr in trs:\n",
    "            name = tr.select_one('td:nth-child(2)').text\n",
    "            quant = tr.select_one('td:nth-child(7)').text\n",
    "            ask_buy = tr.select_one('td:nth-child(8)').text\n",
    "            \n",
    "\n",
    "            # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리\n",
    "            if quant != 'N/A' and ask_buy != 'N/A':\n",
    "                quant = float(quant.replace(',', ''))\n",
    "                ask_buy = float(ask_buy.replace(',', ''))\n",
    "                print(name, quant, ask_buy)\n",
    "                # 엑셀 추가\n",
    "                data.append([name, quant, ask_buy])\n",
    "                \n",
    "        # 요청 간격 추가로 서버 부하 방지\n",
    "        time.sleep(1.5)\n",
    "\n",
    "    except Exception as e:\n",
    "        print(f\"페이지 {page} 처리 중 오류 발생: {str(e)}\")\n",
    "        continue\n",
    "\n",
    "# 데이터프레임 처리 \n",
    "df = pd.DataFrame(data, columns=[\"종목명\", \"거래량\", \"매수호가\"]).dropna()\n",
    "\n",
    "\n",
    "# 파일 저장 최적화\n",
    "timestamp = time.strftime(\"%Y%m%d_%H%M%S\")\n",
    "with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:\n",
    "    df.to_excel(writer, index=False)\n",
    "    # 엑셀 서식 자동 적용\n",
    "    workbook = writer.book\n",
    "    worksheet = writer.sheets['Sheet1']\n",
    "    number_format = workbook.add_format({'num_format': '#,##0.00'})\n",
    "    worksheet.set_column('B:E', 15, number_format)\n",
    "\n",
    "print(f\"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초\")\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ GUI 만들기\n",
    "---\n",
    "#### \"(전체 코드 선택 후, Ctrl+k) 이 코드를 꼼꼼하게 읽고, pyside6를 활용하여 GUI를 만들어줘.\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import pandas as pd\n",
    "import time\n",
    "from requests.adapters import HTTPAdapter\n",
    "from urllib3.util.retry import Retry\n",
    "from PySide6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, \n",
    "                              QHBoxLayout, QLabel, QLineEdit, QPushButton, \n",
    "                              QProgressBar, QTableWidget, QTableWidgetItem, \n",
    "                              QHeaderView, QMessageBox, QFileDialog)\n",
    "from PySide6.QtCore import Qt, QThread, Signal\n",
    "\n",
    "# 웹 크롤링 작업을 위한 스레드 클래스\n",
    "class CrawlerThread(QThread):\n",
    "    \"\"\"\n",
    "    백그라운드에서 웹 크롤링을 수행하는 스레드 클래스\n",
    "    GUI가 멈추지 않도록 별도 스레드로 실행\n",
    "    \"\"\"\n",
    "    # 시그널 정의\n",
    "    progress_signal = Signal(int)  # 진행 상황 업데이트\n",
    "    data_signal = Signal(list)     # 수집된 데이터 전달\n",
    "    error_signal = Signal(str)     # 오류 메시지 전달\n",
    "    finished_signal = Signal(int, float)  # 완료 시그널 (데이터 수, 소요 시간)\n",
    "\n",
    "    def __init__(self, pages):\n",
    "        super().__init__()\n",
    "        self.pages = pages\n",
    "\n",
    "    def run(self):\n",
    "        # 세션 설정으로 성능 향상 및 안정성 확보\n",
    "        session = requests.Session()\n",
    "        retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])\n",
    "        session.mount('https://', HTTPAdapter(max_retries=retries))\n",
    "\n",
    "        # 웹 크롤링 차단 방지를 위한 헤더 설정\n",
    "        headers = {\n",
    "            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'\n",
    "        }\n",
    "\n",
    "        data = []\n",
    "        start_time = time.time()\n",
    "\n",
    "        for page in range(1, self.pages+1):\n",
    "            try:\n",
    "                url = f\"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe\"\n",
    "                # 효율적인 URL 구성 및 요청\n",
    "                response = session.get(\n",
    "                    url,\n",
    "                    headers=headers,\n",
    "                    timeout=10\n",
    "                )\n",
    "                response.raise_for_status()\n",
    "                \n",
    "                soup = BeautifulSoup(response.content, 'html.parser')\n",
    "                trs = soup.select(\"#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']\")\n",
    "                \n",
    "                for tr in trs:\n",
    "                    name = tr.select_one('td:nth-child(2)').text\n",
    "                    market_sum = tr.select_one('td:nth-child(7)').text\n",
    "                    debt_total = tr.select_one('td:nth-child(8)').text\n",
    "                    sales_increasing_rate = tr.select_one('td:nth-child(9)').text\n",
    "                    frgn_rate = tr.select_one('td:nth-child(10)').text\n",
    "                    per = tr.select_one('td:nth-child(11)').text\n",
    "                    roe = tr.select_one('td:nth-child(12)').text\n",
    "\n",
    "                    # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리\n",
    "                    if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':\n",
    "                        market_sum = float(market_sum.replace(',', ''))\n",
    "                        debt_total = float(debt_total.replace(',', ''))\n",
    "                        sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))\n",
    "                        frgn_rate = float(frgn_rate.replace(',', ''))\n",
    "                        per = float(per.replace(',', ''))\n",
    "                        roe = float(roe.replace(',', ''))\n",
    "                        # 데이터 추가\n",
    "                        data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])\n",
    "                \n",
    "                # 진행 상황 업데이트\n",
    "                progress = int((page / self.pages) * 100)\n",
    "                self.progress_signal.emit(progress)\n",
    "                \n",
    "                # 요청 간격 추가로 서버 부하 방지\n",
    "                time.sleep(1.5)\n",
    "\n",
    "            except Exception as e:\n",
    "                self.error_signal.emit(f\"페이지 {page} 처리 중 오류 발생: {str(e)}\")\n",
    "                continue\n",
    "\n",
    "        # 수집 완료 후 데이터 전송\n",
    "        self.data_signal.emit(data)\n",
    "        self.finished_signal.emit(len(data), time.time() - start_time)\n",
    "\n",
    "\n",
    "class StockCrawlerApp(QMainWindow):\n",
    "    \"\"\"\n",
    "    주식 데이터 크롤링 및 분석을 위한 메인 GUI 애플리케이션\n",
    "    \"\"\"\n",
    "    def __init__(self):\n",
    "        super().__init__()\n",
    "        self.df = None  # 데이터프레임 저장 변수\n",
    "        self.initUI()\n",
    "        \n",
    "    def initUI(self):\n",
    "        # 메인 윈도우 설정\n",
    "        self.setWindowTitle('주식 데이터 수집기')\n",
    "        self.setGeometry(100, 100, 1000, 600)\n",
    "        \n",
    "        # 중앙 위젯 및 레이아웃 설정\n",
    "        central_widget = QWidget()\n",
    "        self.setCentralWidget(central_widget)\n",
    "        main_layout = QVBoxLayout(central_widget)\n",
    "        \n",
    "        # 입력 영역 레이아웃\n",
    "        input_layout = QHBoxLayout()\n",
    "        \n",
    "        # 페이지 수 입력\n",
    "        input_layout.addWidget(QLabel('수집할 페이지 수:'))\n",
    "        self.page_input = QLineEdit('1')\n",
    "        self.page_input.setFixedWidth(100)\n",
    "        input_layout.addWidget(self.page_input)\n",
    "        \n",
    "        # 수집 버튼\n",
    "        self.crawl_button = QPushButton('데이터 수집 시작')\n",
    "        self.crawl_button.clicked.connect(self.start_crawling)\n",
    "        input_layout.addWidget(self.crawl_button)\n",
    "        \n",
    "        # 저장 버튼\n",
    "        self.save_button = QPushButton('엑셀로 저장')\n",
    "        self.save_button.clicked.connect(self.save_to_excel)\n",
    "        self.save_button.setEnabled(False)  # 초기에는 비활성화\n",
    "        input_layout.addWidget(self.save_button)\n",
    "        \n",
    "        input_layout.addStretch()\n",
    "        main_layout.addLayout(input_layout)\n",
    "        \n",
    "        # 진행 상황 표시\n",
    "        progress_layout = QHBoxLayout()\n",
    "        progress_layout.addWidget(QLabel('진행 상황:'))\n",
    "        self.progress_bar = QProgressBar()\n",
    "        progress_layout.addWidget(self.progress_bar)\n",
    "        main_layout.addLayout(progress_layout)\n",
    "        \n",
    "        # 상태 메시지\n",
    "        self.status_label = QLabel('준비됨')\n",
    "        main_layout.addWidget(self.status_label)\n",
    "        \n",
    "        # 데이터 테이블\n",
    "        self.table = QTableWidget()\n",
    "        self.table.setColumnCount(7)\n",
    "        self.table.setHorizontalHeaderLabels([\"종목명\", \"시가총액(억)\", \"부채총계(억)\", \n",
    "                                             \"매출증가율\", \"외국인비율\", \"PER(배)\", \"ROE(%)\"])\n",
    "        self.table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)\n",
    "        main_layout.addWidget(self.table)\n",
    "        \n",
    "    def start_crawling(self):\n",
    "        \"\"\"크롤링 작업 시작\"\"\"\n",
    "        try:\n",
    "            pages = int(self.page_input.text())\n",
    "            if pages <= 0:\n",
    "                QMessageBox.warning(self, '입력 오류', '페이지 수는 1 이상이어야 합니다.')\n",
    "                return\n",
    "                \n",
    "            # UI 상태 업데이트\n",
    "            self.crawl_button.setEnabled(False)\n",
    "            self.save_button.setEnabled(False)\n",
    "            self.progress_bar.setValue(0)\n",
    "            self.status_label.setText('데이터 수집 중...')\n",
    "            self.table.setRowCount(0)\n",
    "            \n",
    "            # 크롤링 스레드 시작\n",
    "            self.crawler_thread = CrawlerThread(pages)\n",
    "            self.crawler_thread.progress_signal.connect(self.update_progress)\n",
    "            self.crawler_thread.data_signal.connect(self.display_data)\n",
    "            self.crawler_thread.error_signal.connect(self.show_error)\n",
    "            self.crawler_thread.finished_signal.connect(self.crawling_finished)\n",
    "            self.crawler_thread.start()\n",
    "            \n",
    "        except ValueError:\n",
    "            QMessageBox.warning(self, '입력 오류', '유효한 페이지 수를 입력하세요.')\n",
    "    \n",
    "    def update_progress(self, value):\n",
    "        \"\"\"진행 상황 업데이트\"\"\"\n",
    "        self.progress_bar.setValue(value)\n",
    "    \n",
    "    def display_data(self, data):\n",
    "        \"\"\"수집된 데이터 테이블에 표시\"\"\"\n",
    "        # 데이터프레임 생성\n",
    "        self.df = pd.DataFrame(data, columns=[\"종목명\", \"시가총액(억)\", \"부채총계(억)\", \n",
    "                                             \"매출증가율\", \"외국인비율\", \"PER(배)\", \"ROE(%)\"])\n",
    "        \n",
    "        # 테이블에 데이터 표시\n",
    "        self.table.setRowCount(len(data))\n",
    "        for row_idx, row_data in enumerate(data):\n",
    "            for col_idx, cell_data in enumerate(row_data):\n",
    "                item = QTableWidgetItem()\n",
    "                # 숫자 데이터는 오른쪽 정렬, 문자열은 왼쪽 정렬\n",
    "                if col_idx == 0:  # 종목명\n",
    "                    item.setText(str(cell_data))\n",
    "                    item.setTextAlignment(Qt.AlignLeft | Qt.AlignVCenter)\n",
    "                else:  # 숫자 데이터\n",
    "                    item.setText(f\"{cell_data:,.2f}\")\n",
    "                    item.setTextAlignment(Qt.AlignRight | Qt.AlignVCenter)\n",
    "                self.table.setItem(row_idx, col_idx, item)\n",
    "    \n",
    "    def show_error(self, error_msg):\n",
    "        \"\"\"오류 메시지 표시\"\"\"\n",
    "        self.status_label.setText(f\"오류: {error_msg}\")\n",
    "    \n",
    "    def crawling_finished(self, data_count, elapsed_time):\n",
    "        \"\"\"크롤링 작업 완료 처리\"\"\"\n",
    "        self.crawl_button.setEnabled(True)\n",
    "        self.save_button.setEnabled(True)\n",
    "        self.status_label.setText(f\"총 {data_count}개 데이터 수집 완료 | 소요 시간: {elapsed_time:.2f}초\")\n",
    "    \n",
    "    def save_to_excel(self):\n",
    "        \"\"\"데이터를 엑셀 파일로 저장\"\"\"\n",
    "        if self.df is None or self.df.empty:\n",
    "            QMessageBox.warning(self, '저장 오류', '저장할 데이터가 없습니다.')\n",
    "            return\n",
    "            \n",
    "        # 파일 저장 다이얼로그\n",
    "        timestamp = time.strftime(\"%Y%m%d_%H%M%S\")\n",
    "        default_filename = f'stock_data_{timestamp}.xlsx'\n",
    "        file_path, _ = QFileDialog.getSaveFileName(\n",
    "            self, '엑셀 파일 저장', default_filename, 'Excel Files (*.xlsx)')\n",
    "            \n",
    "        if file_path:\n",
    "            try:\n",
    "                # 엑셀 파일 저장\n",
    "                with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:\n",
    "                    self.df.to_excel(writer, index=False)\n",
    "                    # 엑셀 서식 자동 적용\n",
    "                    workbook = writer.book\n",
    "                    worksheet = writer.sheets['Sheet1']\n",
    "                    number_format = workbook.add_format({'num_format': '#,##0.00'})\n",
    "                    worksheet.set_column('B:G', 15, number_format)\n",
    "                \n",
    "                QMessageBox.information(self, '저장 완료', f'데이터가 성공적으로 저장되었습니다.\\n{file_path}')\n",
    "            except Exception as e:\n",
    "                QMessageBox.critical(self, '저장 오류', f'파일 저장 중 오류가 발생했습니다: {str(e)}')\n",
    "\n",
    "\n",
    "if __name__ == '__main__':\n",
    "    app = QApplication(sys.argv)\n",
    "    window = StockCrawlerApp()\n",
    "    window.show()\n",
    "    sys.exit(app.exec())\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 시가총액 분석 시 효과적으로 활용할 항목 및 상세 분석법\n",
    "\n",
    "## 1. 시가총액(억)\n",
    "- **정의**: 기업의 전체 주식 수에 현재 주가를 곱한 값으로 기업의 전체 시장가치를 나타냄.\n",
    "- **활용법**:\n",
    "  - 기업의 규모와 시장 영향력을 평가할 수 있으며, 산업 내 경쟁사와의 상대적 비교를 통해 안정성을 판단함.\n",
    "  - 규모가 클수록 안정적이지만, 성장률은 다소 낮을 수 있음.\n",
    "\n",
    "## 2. 외국인비율\n",
    "- **정의**: 기업 주식을 보유한 외국인 투자자의 비율.\n",
    "- **활용법**:\n",
    "  - 외국인 투자자의 관심도와 신뢰도를 파악하여 글로벌 시장에서의 기업 평가를 판단.\n",
    "  - 외국인 비율 증가 시 주가 상승 가능성을 예측할 수 있으며, 급격한 변동은 리스크 신호일 수 있음.\n",
    "\n",
    "## 3. PER(배)\n",
    "- **정의**: 주가수익비율로, 현재 주가가 주당 순이익(EPS)에 비해 몇 배인지를 나타냄.\n",
    "- **활용법**:\n",
    "  - 기업이 고평가 또는 저평가되어 있는지를 판단하는 지표.\n",
    "  - 산업 평균 PER과 비교해 기업의 상대적 가치 평가를 수행할 수 있음.\n",
    "\n",
    "## 4. ROE(%)\n",
    "- **정의**: 자기자본이익률로 자기자본 대비 순이익 비율.\n",
    "- **활용법**:\n",
    "  - 기업의 경영 효율성과 수익성을 평가하여 투자 수익의 지속가능성을 판단.\n",
    "  - ROE가 높은 기업은 일반적으로 자본 효율성이 뛰어나며 장기적으로 주가가 안정적으로 상승할 가능성이 높음.\n",
    "\n",
    "## 5. 부채총계(억)\n",
    "- **정의**: 기업이 가지고 있는 모든 부채의 총합.\n",
    "- **활용법**:\n",
    "  - 기업의 재무 안정성과 부실 가능성을 분석하는 주요 지표.\n",
    "  - 부채 비율이 높으면 경기 침체 시 기업의 재무 상태가 빠르게 악화될 수 있어 리스크 관리에 필수적.\n",
    "\n",
    "## 6. 매출액증가율\n",
    "- **정의**: 전년 대비 매출 증가율.\n",
    "- **활용법**:\n",
    "  - 기업의 성장 잠재력과 시장 경쟁력을 직접적으로 반영.\n",
    "  - 꾸준한 매출 성장률은 장기적으로 기업 가치 상승을 의미하며, 투자자에게 긍정적인 지표.\n",
    "\n",
    "---\n",
    "\n",
    "## 추가적인 심층 분석법\n",
    "\n",
    "### 시가총액 대비 거래대금 비율\n",
    "- 거래대금이 높으면 유동성이 좋고, 가격 변동성도 큼.\n",
    "- 단기 매매나 주가 상승 가능성을 평가할 때 유용.\n",
    "\n",
    "### 자산총계 대비 부채총계 비율(부채비율)\n",
    "- 부채비율이 낮으면 재무 안정성이 높고, 장기적인 투자에 유리.\n",
    "- 부채비율이 높으면 재무 위험이 증가하므로 투자 시 유의.\n",
    "\n",
    "### 당기순이익과 주당순이익(EPS)\n",
    "- 기업의 실제 수익 창출 능력을 평가.\n",
    "- 꾸준히 증가하는 기업은 장기적으로 투자 가치가 높음.\n",
    "\n",
    "---\n",
    "\n",
    "## 추가 활용할 수 있는 분석 지표 및 방법\n",
    "- **ROE 및 ROA** 분석으로 자본 효율성을 평가.\n",
    "- **주당순이익(EPS)** 성장률을 통한 기업의 실질적 수익성 분석.\n",
    "- 산업별 평균 지표와의 비교 분석을 통해 상대적인 평가.\n",
    "\n",
    "---\n",
    "\n",
    "## 심화 분석을 위한 질문\n",
    "- 외국인 투자자의 순매수 추이를 이용하여 주가 움직임을 예측할 수 있는 분석 방법 탐색.\n",
    "- 매출액증가율과 PER을 동시에 고려하여 기업의 성장성과 가치 평가를 통합적으로 판단할 수 있는 방법 연구.\n",
    "\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "base",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}